In [2]:
# CLIENT

import socket
import pickle
import cloudpickle

host = '127.0.0.1'         # Canalul de comunicare / localhost
port = 8881                # Portul pe care se asculta

# Se face legatura intre CLIENT si SERVER.
clientSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
clientSocket.connect((host, port))

def startClient():
    # Clientul ii da semnal serverului ca vrea sa inceapajocul la comanda START.
    # In caz contrat, clientul primeste de la server un mesaj sub forma unei erori.
    stareJoc = input("Incepem jocul? (START) >>> ")
    clientSocket.send(stareJoc.encode())
    stareJocServer = clientSocket.recv(4096)

    # Odata inceput jocul, clientul primeste tabelul sub forma unei functii.
    if stareJocServer.decode() == "start":
        punctajRecv = clientSocket.recv(4096)
        punctaj = pickle.loads(punctajRecv)
        punctaj()

        # Variabile folosie pentru a iesi din while-uri, atunci cand este cazul.
        stareJocServer2 = stareJocServer
        stareJocServer3 = stareJocServer

        while stareJocServer2.decode() == "start":      
            comanda = input("Comanda >1> ")
            clientSocket.send(comanda.encode())
            comandaServer = clientSocket.recv(4096)

            stareJocServer3 = stareJocServer2

            if comandaServer.decode() == "arunca":
                zaruriRecv = clientSocket.recv(4096)
                zaruri = pickle.loads(zaruriRecv)

                rRecv = clientSocket.recv(4096)
                print(zaruri, "R =", int(rRecv.decode()))

                while stareJocServer3.decode() == "start":
                    comanda2 = input("Comanda >2> ")
                    clientSocket.send(comanda2.encode())
                    comandaServer2 = clientSocket.recv(4096)

                    # Odata ales locul din tabel, noul punctaj este afisat clientului.
                    # In cazul in care locul este deja ocupat, serverul percepe comanda drept una eronata, indemnand clientul sa faca o alta alegere.
                    # Totodata, jocul este incheiar pentru client atunci cand serverul percepe toate locurile ca find ocupate.
                    if comandaServer2.decode() == "N1" or comandaServer2.decode() == "N2" or comandaServer2.decode() == "N3" or comandaServer2.decode() == "N4" or comandaServer2.decode() == "N5" or comandaServer2.decode() == "N6" or comandaServer2.decode() == "JOKER" or comandaServer2.decode() == "TRIPLA" or comandaServer2.decode() == "CHINTA" or comandaServer2.decode() == "FULL" or comandaServer2.decode() == "CAREU" or comandaServer2.decode() == "YAMS":
                        mesajJocTerminat = clientSocket.recv(4096)
                        if mesajJocTerminat.decode() == "continua":

                            punctajRecv = clientSocket.recv(4096)
                            punctaj = pickle.loads(punctajRecv)
                            punctaj()
                            break
                        else:
                            print("\n", mesajJocTerminat.decode())

                            punctajRecv = clientSocket.recv(4096)
                            punctaj = pickle.loads(punctajRecv)
                            punctaj()

                            stareJocServer3 = comandaServer2
                            stareJocServer2 = comandaServer2
                            break

                    # Clientul primeste de la server noile zaruri.
                    elif comandaServer2.decode() == "keep":
                        zaruriNoiRecv = clientSocket.recv(4096)
                        zaruriNoi = pickle.loads(zaruriNoiRecv)
                        rRecv = clientSocket.recv(4096)

                        print(zaruriNoi, "R =", int(rRecv.decode()))

                    # Clientul primeste tabelul sub forma unei functii, atunci cand doreste.
                    elif comandaServer2.decode() == "punctaj":

                        punctajRecv = clientSocket.recv(4096)
                        punctaj = pickle.loads(punctajRecv)
                        punctaj()

                    # Daca clinetul cere abandonul, jocul se incheie.
                    elif comandaServer2.decode() == "abandon":
                        print("Joc Abandonat \n")
                        stareJocServer3 = comandaServer2
                        stareJocServer2 = comandaServer2
                        break
                    else:
                        print(comandaServer2.decode())

            # Clientul primeste tabelul sub forma unei functii, atunci cand doreste.
            elif comandaServer.decode() == "punctaj":
                punctajRecv = clientSocket.recv(4096)
                punctaj = pickle.loads(punctajRecv)
                punctaj()

            # Daca clinetul cere abandonul, jocul se incheie.
            elif comandaServer.decode() == "abandon":
                print("Joc Abandonat \n")
                break
            # In cazul in care clientul greseste o comanda, serverul o semnaleaza ca find eronata.
            else:
                print(comandaServer.decode())
    # In cazul in care clientul greseste o comanda, serverul o semnaleaza ca find eronata.
    else:
        print(stareJocServer.decode())


    clientSocket.close()

startClient()

Incepem jocul? (START) >>> START


N1 ----->  
N2 ----->  
N3 ----->  
N4 ----->  
N5 ----->  
N6 ----->  
BONUS -->  
JOKER -->  
TRIPLA ->  
CHINTA ->  
FULL --->  
CAREU -->  
YAMS --->  
TOTAL -->  


Comanda >1> ARUCA
COMANDA ERONATA
Comanda >1> ARUNCA
[1, 2, 3, 5, 6] R = 2
Comanda >2> KEEP(1,2,3)
[1, 2, 3, 3, 6] R = 1
Comanda >2> KEEP(1,2,3)
[1, 2, 3, 4, 6] R = 0
Comanda >2> ABANDON
Joc Abandonat 

